In [4]:
import numpy as np
import pandas as pd
import glob 
import os 
import matplotlib.pyplot as plt
import random as rand
from pathlib import Path, PureWindowsPath
import subprocess



In [5]:
def Vec(theta, phi): 
    vec = np.array([np.cos(phi)*np.sin(theta), np.sin(theta)*np.sin(phi), np.cos(theta)])
    return vec

In [58]:
def B_Field_Range(K, M_sat, bMax, bMin, bStep):
    '''Takes K and Msat and returns the anisotropy field'''
    bAnis = (2*K/(M_sat))*1e3
    n_steps = int((bMax - bMin)/(2*bStep))
#     bMin = bMin*1e-3
#     bMax = bMax*1e-3
#     print(bAnis, bMin, bMax)
    if bAnis >= bMax or bAnis <= bMin:
        full_range = np.linspace(bMin, bMax, 2*n_steps)
    else:
        lower_range = [round(x,2) for x in np.linspace(bMin, bAnis, n_steps)]
        upper_range = [round(x,2) for x in np.linspace(bAnis, bMax, n_steps)]
        full_range = np.append(lower_range, upper_range)
    return full_range
    

In [59]:
print(B_Field_Range(10000, 800000, 2000,0,10))

[  0.00000000e+00   2.50000000e-01   5.10000000e-01   7.60000000e-01
   1.01000000e+00   1.26000000e+00   1.52000000e+00   1.77000000e+00
   2.02000000e+00   2.27000000e+00   2.53000000e+00   2.78000000e+00
   3.03000000e+00   3.28000000e+00   3.54000000e+00   3.79000000e+00
   4.04000000e+00   4.29000000e+00   4.55000000e+00   4.80000000e+00
   5.05000000e+00   5.30000000e+00   5.56000000e+00   5.81000000e+00
   6.06000000e+00   6.31000000e+00   6.57000000e+00   6.82000000e+00
   7.07000000e+00   7.32000000e+00   7.58000000e+00   7.83000000e+00
   8.08000000e+00   8.33000000e+00   8.59000000e+00   8.84000000e+00
   9.09000000e+00   9.34000000e+00   9.60000000e+00   9.85000000e+00
   1.01000000e+01   1.03500000e+01   1.06100000e+01   1.08600000e+01
   1.11100000e+01   1.13600000e+01   1.16200000e+01   1.18700000e+01
   1.21200000e+01   1.23700000e+01   1.26300000e+01   1.28800000e+01
   1.31300000e+01   1.33800000e+01   1.36400000e+01   1.38900000e+01
   1.41400000e+01   1.43900000e+01

In [61]:
def FMR_Thin_Film_Filemaker(bInPlane=True, anisVec='In_Plane', axis='easy', 
                            bMax=2000, bMin=0, bStep=10, kMax=100000, kMin=0, kStep=10000, ratio=1,
                            PBC=[0,0,0], gridsize=[512,512,4], cellsize=[4,4,4], Msat=8e5, Aex=1.3e-11, alpha=0.008,
                            solver=5, tStep=5e-13, runtime=1e-9):
    '''
    bInPlane = bool,              True if B field is in the plane of the film
    inducdAnis = bool,            True if uniaxial anisotropy is applied
    anisVec = string,              'InPlane', Out_of_Plane', or 'Angular'
    axis = string,                 'easy' or 'hard', Rotates the magnetic field in x-y plane from <100> to <010>.
                                        ROTATES ANISOTROPY WHEN ANISVEC=='ANGULAR' and BINPANE==FALSE. 
    bMax, bMin, bStep = floats,      Magnetic Field strength in milli-tesla
    kMax, kMin, kStep = floats,      Anisotropy Strength in J/m^3
    ratio = integer,                Scales the y-direction of the thin film by inverse of input
    PBC = [list of length 3],      Periodic Boundary Conditions, number of periodic repetitions in each direction
    gridsize = [list of length 3], Number of cells in each direction
    cellsize = [list of length 3], Side lengths (in meters) of a cuboidal cell
    Msat = float,                   Saturation Magnetization in J/m^3
    Aex = float,                    Exchange constant in J/m
    alpha = float,                  Damping constant (unitless)
    solver  = int,                  Type of solver used by Mumax (1:Euler, 2:Adaptive Heun, 3:RK23 (Bogacki-Shampine),
                                        4:Classical 4th-order Runge-Kutta, 5:RK45 (Dormand-Prince))
    tStep = float,                  Timestep (seconds), important for Fourier Transform
    runtime = float,                Length of simulation (seconds), choose to be just the region ofexponetial decay
    '''
    basePath = os.getcwd()
#     print(basePath)
    angles = np.array([0])
    anisPhi = 0
#     path = r'C:/Users/Rob/Desktop/Research/FMR_paper/'
#     '''In plane field'''
    if bInPlane:
        fieldPath = 'In_Plane_Field'
        theta = np.pi/2
#     '''Out of Plane Field'''

    else:
        fieldPath = 'Out_of_Plane_Field'
        B_vec = np.array([0,0,1])
        theta = rand.randrange(1,10)*1e-2
        phi = rand.randrange(1,10)*1e-2
        B0_vec = Vec(theta,phi)
#     
#     
    if anisVec == 'In_Plane':
        anisPath = 'In_Plane_Anis'
        theta_K = np.pi/2
        phi_K = 0
#         K_vec = np.array([1,0,0])
        if axis =='easy':
            axisPath = 'EasyAxis'
            B_vec = np.array([1,0,0])
            theta = np.pi/2 - rand.randrange(1,10)*1e-2
            phi = rand.randrange(1,10)*1e-2
            B0_vec = Vec(theta,phi)
        else:
            axisPath = 'HardAxis'
            B_vec = [0,1,0]
            theta = np.pi/2 - rand.randrange(1,10)*1e-2
            phi = np.pi/2 - rand.randrange(1,10)*1e-2
            B0_vec = Vec(theta,phi)
  
    elif anisVec == 'Out_of_Plane':
        anisPath = 'Out_of_Plane_Anis'
        theta_K = 0
        phi_K = 0
#         K_vec = np.array([0,0,1])
        if axis =='easy':
            axisPath = 'EasyAxis'
            B_vec = np.array([1,0,0])
            theta = np.pi/2 - rand.randrange(1,10)*1e-2
            phi = rand.randrange(1,10)*1e-2
            B0_vec = Vec(theta,phi)
        else:
            axisPath = 'HardAxis'
            B_vec = [0,1,0]
            theta = np.pi/2 - rand.randrange(1,10)*1e-2
            phi = np.pi/2 - rand.randrange(1,10)*1e-2
            B0_vec = Vec(theta,phi)
   
    elif anisVec == 'Angular':
        anisPath = 'Angular_Anis'
        angles = np.linspace(0,2*np.pi,180)
        
        if bInPlane:
            
            '''sweep K in x-y direction'''
            phi_K = angles
            theta_K = np.pi/2*np.ones(len(angles))
            theta = np.pi/2 - rand.randrange(1,10)*1e-2

            if axis =='easy':
                axisPath = 'EasyAxis'
#                 K_vec = np.array([np.cos(angle),np.sin(angle),0])
                B_vec = np.array([1,0,0])
#                 theta = np.pi/2 - rand.randrange(1,10)*1e-2
                phi = rand.randrange(1,10)*1e-2
                B0_vec = Vec(theta,phi)
                
            else:
                axisPath = 'HardAxis'
#                 K_vec = np.array([np.cos(angle), np.sin(angle),0])
                B_vec = np.array([0,1,0])
#                 theta = np.pi/2 - rand.randrange(1,10)*1e-2
                phi = np.pi/2 - rand.randrange(1,10)*1e-2
                B0_vec = Vec(theta,phi)

        else:
            '''sweep K in z direction'''
            theta_K = angles
            
            B_vec = np.array([0,0,1])
            
            theta = rand.randrange(1,10)*1e-2
            phi = np.pi/2 - rand.randrange(1,10)*1e-2
            B0_vec = Vec(theta,phi)
    
            if axis =='easy':
                axisPath = 'EasyAxis'
                phi_K = np.zeros(len(theta_K))
#                 K_vec = np.array([np.sin(angle),0,np.cos(angle)])
                
            else:
                axisPath = 'HardAxis'
                phi_K = np.pi/2*np.ones(len(theta_K))
#                 K_vec = np.array([0,np.sin(angle),np.cos(angle)])
    
    
    else:
        print('''Error: Unexpected String for anisVec, 
                expects: "InPlane", "Out_of_Plane", or "Angular"''')


    K_strength = np.arange(kMin, kMax+1, kStep)
    K_vec_array = []
    if not hasattr(theta_K, '__iter__'):
        theta_K = [theta_K]
        phi_K = [phi_K]
    for i, a in enumerate(zip(theta_K,phi_K)): 
#         print(i, a)
        K_vec = Vec(a[0],a[1])
#         print(K_vec)
        K_vec_array.append(K_vec)

#     for i, K in enumerate(K_vecs):
#         print(i,K)
#     print (K_vec_array)
    
    B_fields = np.arange(bMax,bMin-1,-bStep)
    
    
    folderpath = os.path.join(basePath, fieldPath, anisPath, axisPath)
#     print(folderpath)
    bashCommand = "mkdir -p " + folderpath
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()

    count = 0 
    
    
    for K in K_strength:

        for B in B_Field_Range(K, Msat, bMax, bMin, bStep):

    
            for i, K_vec in enumerate(K_vec_array):
                try:
                    _ = K_vec[0]
                except:
                    K_vec = K_vec_array
#                 print(i, K_vec[0], theta_K[i], phi_K[i])
                count += 1
                lines = [
    #                 "SetMesh(512,512/{1},4, {0}e-09,{0}e-09,{0}e-09, 4,4,0);".format(cellsize,ratio),
                    'SetPBC({0}, {1}, {2});'.format(PBC[0],PBC[1],PBC[2]),        
                    'SetGridSize({0}, {1}, {2});'.format(gridsize[0], gridsize[1]/ratio, gridsize[2]),
                    'SetCellSize({0}, {1}, {2});'.format(cellsize[0], cellsize[1], cellsize[2]),
                    'Msat  = {};'.format(Msat),
                    'Aex   = {};'.format(Aex),
                    'alpha = {};'.format(alpha),
                    'edgesmooth = 8;',
                    'B := {}e-3;'.format(B),
                    'anisU = vector({0}, {1}, {2});'.format(K_vec[0], K_vec[1], K_vec[2]),
                    'Ku1 = {};'.format(K),
                    'setSolver({});'.format(solver),
                    'MaxDt = {};'.format(tStep),
                    'TableAdd(Ku1)',
                    'TableAdd(B_ext);',
                    'TableAutoSave({});'.format(runtime),
    #                         '{}'.format(Recorder),


                    'm = uniform({0}, {1}, {2});'.format(B_vec[0], B_vec[1], B_vec[2]),
                    'B_ext = vector(B*{0}, B*{1}, B*{2});'.format(B0_vec[0], B0_vec[1], B0_vec[2]),
                    'relax();',
                    'B_ext = vector(B*{0}, B*{1}, B*{2});'.format(B_vec[0], B_vec[1], B_vec[2]),
                    'run({});'.format(runtime),
                ]
                
                # Name the files
                filename = 'ThinFilmFMR_'
                if bInPlane:
                    filename += 'BInPlane({})_'.format(B)
                else:
                    filename += 'BOutOfPlane({})_'.format(B)
                
#                 if inducedAnis:
#                     filename = 'InducedAnis({})_'.format(K)
#                 else:
#                     filename = 'ShapeAnis({})_'.format(K)

                if anisVec == 'Angular':
                    filename += anisVec + 'Anis_K({})_Theta({:.5f})_Phi({:.5f})_'.format(K, theta_K[i], phi_K[i])
                else:
                    filename += anisVec + 'Anis_K({})_'.format(K)
                filename += 'Ratio({})'.format(ratio)
                filename += '.mx3'
                print(count, filename)

                full_filename = os.path.join(basePath, fieldPath, anisPath, axisPath, filename)

                with open(full_filename, 'w') as f:
                    f.writelines("%s\n" % l for l in lines)



Make In plane field, in plane anis

In [67]:
FMR_Thin_Film_Filemaker(bInPlane=True, anisVec='In_Plane', axis='easy', 
                            bMax=2000, bMin=0, bStep=10, kMax=100000, kMin=0, kStep=10000, ratio=1,
                            PBC=[0,0,0], gridsize=[512,512,4], cellsize=[4,4,4], Msat=8e5, Aex=1.3e-11, alpha=0.008,
                            solver=5, tStep=5e-13, runtime=1e-9)

1 ThinFilmFMR_BInPlane(0.0)_In_PlaneAnis_K(0)_Ratio(1).mx3
2 ThinFilmFMR_BInPlane(10.050251256281408)_In_PlaneAnis_K(0)_Ratio(1).mx3
3 ThinFilmFMR_BInPlane(20.100502512562816)_In_PlaneAnis_K(0)_Ratio(1).mx3
4 ThinFilmFMR_BInPlane(30.150753768844226)_In_PlaneAnis_K(0)_Ratio(1).mx3
5 ThinFilmFMR_BInPlane(40.20100502512563)_In_PlaneAnis_K(0)_Ratio(1).mx3
6 ThinFilmFMR_BInPlane(50.25125628140704)_In_PlaneAnis_K(0)_Ratio(1).mx3
7 ThinFilmFMR_BInPlane(60.30150753768845)_In_PlaneAnis_K(0)_Ratio(1).mx3
8 ThinFilmFMR_BInPlane(70.35175879396985)_In_PlaneAnis_K(0)_Ratio(1).mx3
9 ThinFilmFMR_BInPlane(80.40201005025126)_In_PlaneAnis_K(0)_Ratio(1).mx3
10 ThinFilmFMR_BInPlane(90.45226130653268)_In_PlaneAnis_K(0)_Ratio(1).mx3
11 ThinFilmFMR_BInPlane(100.50251256281408)_In_PlaneAnis_K(0)_Ratio(1).mx3
12 ThinFilmFMR_BInPlane(110.55276381909549)_In_PlaneAnis_K(0)_Ratio(1).mx3
13 ThinFilmFMR_BInPlane(120.6030150753769)_In_PlaneAnis_K(0)_Ratio(1).mx3
14 ThinFilmFMR_BInPlane(130.6532663316583)_In_PlaneAnis_

In [66]:
FMR_Thin_Film_Filemaker(bInPlane=True, anisVec='In_Plane', axis='hard', 
                            bMax=2000, bMin=0, bStep=10, kMax=100000, kMin=0, kStep=10000, ratio=1,
                            PBC=[0,0,0], gridsize=[512,512,4], cellsize=[4,4,4], Msat=8e5, Aex=1.3e-11, alpha=0.008,
                            solver=5, tStep=5e-13, runtime=1e-9)

1 ThinFilmFMR_BInPlane(0.0)_In_PlaneAnis_K(0)_Ratio(1).mx3
2 ThinFilmFMR_BInPlane(10.050251256281408)_In_PlaneAnis_K(0)_Ratio(1).mx3
3 ThinFilmFMR_BInPlane(20.100502512562816)_In_PlaneAnis_K(0)_Ratio(1).mx3
4 ThinFilmFMR_BInPlane(30.150753768844226)_In_PlaneAnis_K(0)_Ratio(1).mx3
5 ThinFilmFMR_BInPlane(40.20100502512563)_In_PlaneAnis_K(0)_Ratio(1).mx3
6 ThinFilmFMR_BInPlane(50.25125628140704)_In_PlaneAnis_K(0)_Ratio(1).mx3
7 ThinFilmFMR_BInPlane(60.30150753768845)_In_PlaneAnis_K(0)_Ratio(1).mx3
8 ThinFilmFMR_BInPlane(70.35175879396985)_In_PlaneAnis_K(0)_Ratio(1).mx3
9 ThinFilmFMR_BInPlane(80.40201005025126)_In_PlaneAnis_K(0)_Ratio(1).mx3
10 ThinFilmFMR_BInPlane(90.45226130653268)_In_PlaneAnis_K(0)_Ratio(1).mx3
11 ThinFilmFMR_BInPlane(100.50251256281408)_In_PlaneAnis_K(0)_Ratio(1).mx3
12 ThinFilmFMR_BInPlane(110.55276381909549)_In_PlaneAnis_K(0)_Ratio(1).mx3
13 ThinFilmFMR_BInPlane(120.6030150753769)_In_PlaneAnis_K(0)_Ratio(1).mx3
14 ThinFilmFMR_BInPlane(130.6532663316583)_In_PlaneAnis_

Make In Plane Field, out of plane anis

In [68]:
FMR_Thin_Film_Filemaker(bInPlane=True, anisVec='Out_of_Plane', axis='easy', 
                            bMax=2000, bMin=0, bStep=10, kMax=100000, kMin=0, kStep=10000, ratio=1,
                            PBC=[0,0,0], gridsize=[512,512,4], cellsize=[4,4,4], Msat=8e5, Aex=1.3e-11, alpha=0.008,
                            solver=5, tStep=5e-13, runtime=1e-9)

1 ThinFilmFMR_BInPlane(0.0)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
2 ThinFilmFMR_BInPlane(10.050251256281408)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
3 ThinFilmFMR_BInPlane(20.100502512562816)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
4 ThinFilmFMR_BInPlane(30.150753768844226)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
5 ThinFilmFMR_BInPlane(40.20100502512563)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
6 ThinFilmFMR_BInPlane(50.25125628140704)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
7 ThinFilmFMR_BInPlane(60.30150753768845)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
8 ThinFilmFMR_BInPlane(70.35175879396985)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
9 ThinFilmFMR_BInPlane(80.40201005025126)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
10 ThinFilmFMR_BInPlane(90.45226130653268)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
11 ThinFilmFMR_BInPlane(100.50251256281408)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
12 ThinFilmFMR_BInPlane(110.55276381909549)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
13 ThinFilmFMR_BInPlane(120.6030150753769)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
14 T

In [69]:
FMR_Thin_Film_Filemaker(bInPlane=True, anisVec='Out_of_Plane', axis='hard', 
                            bMax=2000, bMin=0, bStep=10, kMax=100000, kMin=0, kStep=10000, ratio=1,
                            PBC=[0,0,0], gridsize=[512,512,4], cellsize=[4,4,4], Msat=8e5, Aex=1.3e-11, alpha=0.008,
                            solver=5, tStep=5e-13, runtime=1e-9)

1 ThinFilmFMR_BInPlane(0.0)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
2 ThinFilmFMR_BInPlane(10.050251256281408)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
3 ThinFilmFMR_BInPlane(20.100502512562816)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
4 ThinFilmFMR_BInPlane(30.150753768844226)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
5 ThinFilmFMR_BInPlane(40.20100502512563)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
6 ThinFilmFMR_BInPlane(50.25125628140704)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
7 ThinFilmFMR_BInPlane(60.30150753768845)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
8 ThinFilmFMR_BInPlane(70.35175879396985)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
9 ThinFilmFMR_BInPlane(80.40201005025126)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
10 ThinFilmFMR_BInPlane(90.45226130653268)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
11 ThinFilmFMR_BInPlane(100.50251256281408)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
12 ThinFilmFMR_BInPlane(110.55276381909549)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
13 ThinFilmFMR_BInPlane(120.6030150753769)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
14 T

Out of Plane field, in plane anis

In [70]:
FMR_Thin_Film_Filemaker(bInPlane=False, anisVec='In_Plane', axis='easy', 
                            bMax=2000, bMin=0, bStep=10, kMax=100000, kMin=0, kStep=10000, ratio=1,
                            PBC=[0,0,0], gridsize=[512,512,4], cellsize=[4,4,4], Msat=8e5, Aex=1.3e-11, alpha=0.008,
                            solver=5, tStep=5e-13, runtime=1e-9)

1 ThinFilmFMR_BOutOfPlane(0.0)_In_PlaneAnis_K(0)_Ratio(1).mx3
2 ThinFilmFMR_BOutOfPlane(10.050251256281408)_In_PlaneAnis_K(0)_Ratio(1).mx3
3 ThinFilmFMR_BOutOfPlane(20.100502512562816)_In_PlaneAnis_K(0)_Ratio(1).mx3
4 ThinFilmFMR_BOutOfPlane(30.150753768844226)_In_PlaneAnis_K(0)_Ratio(1).mx3
5 ThinFilmFMR_BOutOfPlane(40.20100502512563)_In_PlaneAnis_K(0)_Ratio(1).mx3
6 ThinFilmFMR_BOutOfPlane(50.25125628140704)_In_PlaneAnis_K(0)_Ratio(1).mx3
7 ThinFilmFMR_BOutOfPlane(60.30150753768845)_In_PlaneAnis_K(0)_Ratio(1).mx3
8 ThinFilmFMR_BOutOfPlane(70.35175879396985)_In_PlaneAnis_K(0)_Ratio(1).mx3
9 ThinFilmFMR_BOutOfPlane(80.40201005025126)_In_PlaneAnis_K(0)_Ratio(1).mx3
10 ThinFilmFMR_BOutOfPlane(90.45226130653268)_In_PlaneAnis_K(0)_Ratio(1).mx3
11 ThinFilmFMR_BOutOfPlane(100.50251256281408)_In_PlaneAnis_K(0)_Ratio(1).mx3
12 ThinFilmFMR_BOutOfPlane(110.55276381909549)_In_PlaneAnis_K(0)_Ratio(1).mx3
13 ThinFilmFMR_BOutOfPlane(120.6030150753769)_In_PlaneAnis_K(0)_Ratio(1).mx3
14 ThinFilmFMR_BO

In [71]:
FMR_Thin_Film_Filemaker(bInPlane=False, anisVec='In_Plane', axis='hard', 
                            bMax=2000, bMin=0, bStep=10, kMax=100000, kMin=0, kStep=10000, ratio=1,
                            PBC=[0,0,0], gridsize=[512,512,4], cellsize=[4,4,4], Msat=8e5, Aex=1.3e-11, alpha=0.008,
                            solver=5, tStep=5e-13, runtime=1e-9)

1 ThinFilmFMR_BOutOfPlane(0.0)_In_PlaneAnis_K(0)_Ratio(1).mx3
2 ThinFilmFMR_BOutOfPlane(10.050251256281408)_In_PlaneAnis_K(0)_Ratio(1).mx3
3 ThinFilmFMR_BOutOfPlane(20.100502512562816)_In_PlaneAnis_K(0)_Ratio(1).mx3
4 ThinFilmFMR_BOutOfPlane(30.150753768844226)_In_PlaneAnis_K(0)_Ratio(1).mx3
5 ThinFilmFMR_BOutOfPlane(40.20100502512563)_In_PlaneAnis_K(0)_Ratio(1).mx3
6 ThinFilmFMR_BOutOfPlane(50.25125628140704)_In_PlaneAnis_K(0)_Ratio(1).mx3
7 ThinFilmFMR_BOutOfPlane(60.30150753768845)_In_PlaneAnis_K(0)_Ratio(1).mx3
8 ThinFilmFMR_BOutOfPlane(70.35175879396985)_In_PlaneAnis_K(0)_Ratio(1).mx3
9 ThinFilmFMR_BOutOfPlane(80.40201005025126)_In_PlaneAnis_K(0)_Ratio(1).mx3
10 ThinFilmFMR_BOutOfPlane(90.45226130653268)_In_PlaneAnis_K(0)_Ratio(1).mx3
11 ThinFilmFMR_BOutOfPlane(100.50251256281408)_In_PlaneAnis_K(0)_Ratio(1).mx3
12 ThinFilmFMR_BOutOfPlane(110.55276381909549)_In_PlaneAnis_K(0)_Ratio(1).mx3
13 ThinFilmFMR_BOutOfPlane(120.6030150753769)_In_PlaneAnis_K(0)_Ratio(1).mx3
14 ThinFilmFMR_BO

Out of plane field, out of plane anis


In [72]:
FMR_Thin_Film_Filemaker(bInPlane=False, anisVec='Out_of_Plane', axis='easy', 
                            bMax=2000, bMin=0, bStep=10, kMax=100000, kMin=0, kStep=10000, ratio=1,
                            PBC=[0,0,0], gridsize=[512,512,4], cellsize=[4,4,4], Msat=8e5, Aex=1.3e-11, alpha=0.008,
                            solver=5, tStep=5e-13, runtime=1e-9)

1 ThinFilmFMR_BOutOfPlane(0.0)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
2 ThinFilmFMR_BOutOfPlane(10.050251256281408)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
3 ThinFilmFMR_BOutOfPlane(20.100502512562816)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
4 ThinFilmFMR_BOutOfPlane(30.150753768844226)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
5 ThinFilmFMR_BOutOfPlane(40.20100502512563)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
6 ThinFilmFMR_BOutOfPlane(50.25125628140704)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
7 ThinFilmFMR_BOutOfPlane(60.30150753768845)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
8 ThinFilmFMR_BOutOfPlane(70.35175879396985)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
9 ThinFilmFMR_BOutOfPlane(80.40201005025126)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
10 ThinFilmFMR_BOutOfPlane(90.45226130653268)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
11 ThinFilmFMR_BOutOfPlane(100.50251256281408)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
12 ThinFilmFMR_BOutOfPlane(110.55276381909549)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
13 ThinFilmFMR_BOutOfPlane(120.6030150753769)_

In [73]:
FMR_Thin_Film_Filemaker(bInPlane=False, anisVec='Out_of_Plane', axis='hard', 
                            bMax=2000, bMin=0, bStep=10, kMax=100000, kMin=0, kStep=10000, ratio=1,
                            PBC=[0,0,0], gridsize=[512,512,4], cellsize=[4,4,4], Msat=8e5, Aex=1.3e-11, alpha=0.008,
                            solver=5, tStep=5e-13, runtime=1e-9)

1 ThinFilmFMR_BOutOfPlane(0.0)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
2 ThinFilmFMR_BOutOfPlane(10.050251256281408)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
3 ThinFilmFMR_BOutOfPlane(20.100502512562816)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
4 ThinFilmFMR_BOutOfPlane(30.150753768844226)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
5 ThinFilmFMR_BOutOfPlane(40.20100502512563)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
6 ThinFilmFMR_BOutOfPlane(50.25125628140704)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
7 ThinFilmFMR_BOutOfPlane(60.30150753768845)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
8 ThinFilmFMR_BOutOfPlane(70.35175879396985)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
9 ThinFilmFMR_BOutOfPlane(80.40201005025126)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
10 ThinFilmFMR_BOutOfPlane(90.45226130653268)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
11 ThinFilmFMR_BOutOfPlane(100.50251256281408)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
12 ThinFilmFMR_BOutOfPlane(110.55276381909549)_Out_of_PlaneAnis_K(0)_Ratio(1).mx3
13 ThinFilmFMR_BOutOfPlane(120.6030150753769)_